In [ ]:
from owslib.csw import CatalogueServiceWeb
from owslib.wms import WebMapService
from owslib.wcs import WebCoverageService

In [ ]:
system_endpoint = 'https://resource-catalogue.185.52.193.87.nip.io/'

In [ ]:
csw = CatalogueServiceWeb(system_endpoint,timeout=30)

In [ ]:
csw.getrecordbyid(id=['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'])

In [ ]:
links = csw.records['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE'].references

In [ ]:
for link in links:
    scheme = link['scheme']
    if 'WMS' in scheme:
        wms_endpoint=link['url']
        print(link['url'])

In [ ]:
wms = WebMapService(wms_endpoint, version='1.3.0')

In [ ]:
list(wms.contents)

In [ ]:
wms['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR'].title

In [ ]:
wms['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR'].boundingBoxWGS84

In [ ]:
[op.name for op in wms.operations]

In [ ]:
wms['S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR'].styles

In [ ]:
wms.getOperationByName('GetMap').formatOptions

In [ ]:
%matplotlib inline
import os, sys
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

def getMap(wms,layerName,bbox,filename):
    wms.getOperationByName('GetMap').formatOptions
    img = wms.getmap(layers=[layerName],
                 size=(600,300),
                 srs='EPSG:4326',
                 bbox=bbox,
                 format='image/png',
                 transparent=True)

    tmpfile = open(filename,'wb')
    tmpfile.write(img.read())
    tmpfile.close()

In [ ]:
getMap(wms,'S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR',(23.4,37.8,24.5,38.8), 'rgb.png')
image1=mpimg.imread('rgb.png')
fig = plt.figure(figsize=(12,7))
img1=plt.imshow(image1,extent=[23.4,24.5,37.8,38.8],aspect='auto')
plt.show()

In [ ]:
wms2 = WebMapService('http://apps.ecmwf.int/wms/?token=public', version='1.3.0')
getMap(wms2,'foreground',(-180,-90,180,90), 'foreground.png')
getMap(wms2,'background',(-180,-90,180,90), 'background.png')
getMap(wms2,'composition_bbaod550',(-180,-90,180,90), 'bbaod550.png')
image1=mpimg.imread('background.png')
image2=mpimg.imread('bbaod550.png')
image3=mpimg.imread('foreground.png')
fig = plt.figure(figsize=(12,7))

img1=plt.imshow(image1,extent=[-180,180,-90,90],aspect='auto')
img2=plt.imshow(image2,extent=[-180,180,-90,90],aspect='auto')
img3=plt.imshow(image3,extent=[-180,180,-90,90],aspect='auto')
plt.show()

### Folium demo

In [ ]:
import os
import folium

print(folium.__version__)

In [ ]:
m = folium.Map(location=[38, 24], zoom_start=6, tiles=None)


folium.raster_layers.TileLayer(
    tiles='http://{s}.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='google',
    name='google maps',
    max_zoom=20,
    subdomains=['mt0', 'mt1', 'mt2', 'mt3'],
    overlay=False,
    control=True,
).add_to(m)

folium.raster_layers.WmsTileLayer(
    url=wms_endpoint,
    layers='S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR',
    name='S2B_MSIL2A_20200902T090559_N0214_R050_T34SGH_20200902T113910.SAFE__TRUE_COLOR',
    fmt='image/png',
    overlay=False,
    control=True,
).add_to(m)

folium.LayerControl().add_to(m)

#m.save(os.path.join('results', 'WMS_and_WMTS.html'))

m

### WCS demo

In [ ]:
for link in links:
    scheme = link['scheme']
    if 'WCS' in scheme:
        wcs_endpoint=link['url'].split('?')[0]
        wcs_id=link['url'].split('eoid=')[1]
        print(link['url'])

In [ ]:
wcs = WebCoverageService(wcs_endpoint, version='2.0.1')

In [ ]:
wcs.contents.keys()